In [3]:
#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

In [2]:
#Primer arcivho csv importado
df1 = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\Individual 1\movies_dataset.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\1350539841.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\Individual 1\movies_dataset.csv")


In [3]:
#Segundo csv importado
df_credits = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\credits.csv")

Proceso de ETL

In [4]:
#Primer paso para desanidar
class desanidar:
    @staticmethod
    def convertir_a_str(valor):
        if isinstance(valor, (list, dict)):
            return json.dumps(valor)
        return str(valor)
    @staticmethod
    def extraer_nombres(valor):
        pattern = r"'name': '([^']*)'"
        coincidencias = re.findall(pattern, valor)
        if len(coincidencias) > 0:
            nombre = coincidencias[0]
            return nombre
        else:
            return None

In [5]:
#Segundo paso para desanidar (Son varios)
df1['belongs_to_collection'] = df1['belongs_to_collection'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [6]:
df1['genres'] = df1['genres'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [7]:
df1['production_companies'] = df1['production_companies'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [8]:
df1['production_countries'] = df1['production_countries'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [9]:
df1['spoken_languages'] = df1['spoken_languages'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [10]:
df_credits['cast'] = df_credits['cast'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [11]:
df_credits['crew'] = df_credits['crew'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [13]:
# Convertir las columnas 'budget' y 'revenue' a tipo numérico
df1['budget'] = pd.to_numeric(df1['budget'], errors='coerce')
df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')

# Calcular el retorno de inversión y asignar el valor 0 cuando no haya datos disponibles
df1['return'] = df1['revenue'].div(df1['budget'], fill_value=0)

In [15]:
##Cambio a numerico el tipo de datos de la columna ID
df1['id'] = pd.to_numeric(df1['id'], errors='coerce')

In [ ]:
#Observo cuales son las columnas sin ID
df1[df1['id'].isnull()]

In [17]:
#Dado que solo son 3, decido eliminarlas
df1 = df1.dropna(subset=['id'])

In [18]:
#Cambio el tipo de dato de float a int de la columna, para que coincida en ambos df
df1['id'] = df1['id'].astype(int)

In [ ]:
#Realizo un merge entre los dos datasets que tenemos
df= pd.merge(df1, df_credits, on='id')

In [64]:
#Cambie nombre de crew a Director
df.rename(columns={'crew':'director'}, inplace=True)


In [65]:
#Cambie nombre de cast a actors
df.rename(columns={'cast':'actors'}, inplace=True)


In [67]:
#Borre la columnas que no voy a utilizar
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df1 = df1.drop(columns=columnas_a_eliminar)

In [ ]:
#Borro una columnas que estaban de mas
df1 = df1.dropna(subset=['release_date'])


In [234]:
#Visualizamos el dataset
df.head(2)

,budget,genres,id,overview,popularity,production_companies,production_countries,release_year,revenue,runtime,...,tagline,title,vote_average,vote_count,return,actors,director,release_month,release_day,dia_espanol
0,30000000.0,Animation,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.7,5415.0,12.451801,Tom Hanks,John Lasseter,octubre,30,Lunes
1,65000000.0,Adventure,8844,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249.0,104.0,...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,Robin Williams,Larry J. Franco,diciembre,15,Viernes


In [ ]:
#Push para guardar el archivo
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)